# Risk Mediation for Heart Failure for BMI through SBP
This notebook is designed to calculate the mediation factors for heart failure SBP. LDL-C and FPG do not directly affect HF in our model, and so do not have a mediation factor here. 

### First, we will load in the relative risk values from GBD


In [1]:
from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
import numpy as np
import pandas as pd

In [21]:
ihd_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best',
                    decomp_step='step4')

ihd_bmi = ihd_bmi.loc[ihd_bmi.cause_id==493].reset_index()
#ihd_bmi = ihd_bmi[['age_group_id','sex_id','draw_0']]
ihd_bmi['age_start'] = np.where(ihd_bmi['age_group_id']==10,25,np.where(ihd_bmi['age_group_id']==11,30,np.where(ihd_bmi['age_group_id']==12,35,np.where(ihd_bmi['age_group_id']==13,40,np.where(ihd_bmi['age_group_id']==14,45,np.where(ihd_bmi['age_group_id']==15,50,np.where(ihd_bmi['age_group_id']==16,55,np.where(ihd_bmi['age_group_id']==17,60,np.where(ihd_bmi['age_group_id']==18,65,np.where(ihd_bmi['age_group_id']==19,70,np.where(ihd_bmi['age_group_id']==20,75,np.where(ihd_bmi['age_group_id']==30,80,np.where(ihd_bmi['age_group_id']==31,85,np.where(ihd_bmi['age_group_id']==32,90,0))))))))))))))
ihd_bmi['sex'] = np.where(ihd_bmi['sex_id']==1,'Male','Female')
#ihd_bmi = ihd_bmi.rename(columns={"draw_0": "ihd_bmi_rr"}).reset_index()
ihd_bmi = ihd_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_bmi.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,135,370,9031,103,2019,14,2,493,1,...,1.595691,1.623079,1.521126,1.594813,1.613120,1.515693,1.563302,417614,45,Female
1,1,136,370,9031,103,2019,17,2,493,1,...,1.502339,1.425549,1.435311,1.433340,1.391671,1.434042,1.474836,417614,60,Female
2,2,137,370,9031,103,2019,12,2,493,1,...,1.529235,1.712504,1.586445,1.631762,1.606061,1.625236,1.634291,417614,35,Female
3,3,138,370,9031,103,2019,13,2,493,1,...,1.693763,1.548311,1.456425,1.512640,1.615120,1.778050,1.562930,417614,40,Female
4,4,139,370,9031,103,2019,20,2,493,1,...,1.194777,1.207121,1.316172,1.282573,1.276711,1.270741,1.282031,417614,75,Female


In [3]:
stroke_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                    age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                    sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_bmi = stroke_bmi.loc[stroke_bmi.cause_id==495].reset_index()
#stroke_bmi = stroke_bmi[['age_group_id','sex_id','draw_0']]
stroke_bmi['age_start'] = np.where(stroke_bmi['age_group_id']==10,25,np.where(stroke_bmi['age_group_id']==11,30,np.where(stroke_bmi['age_group_id']==12,35,np.where(stroke_bmi['age_group_id']==13,40,np.where(stroke_bmi['age_group_id']==14,45,np.where(stroke_bmi['age_group_id']==15,50,np.where(stroke_bmi['age_group_id']==16,55,np.where(stroke_bmi['age_group_id']==17,60,np.where(stroke_bmi['age_group_id']==18,65,np.where(stroke_bmi['age_group_id']==19,70,np.where(stroke_bmi['age_group_id']==20,75,np.where(stroke_bmi['age_group_id']==30,80,np.where(stroke_bmi['age_group_id']==31,85,np.where(stroke_bmi['age_group_id']==32,90,0))))))))))))))
stroke_bmi['sex'] = np.where(stroke_bmi['sex_id']==1,'Male','Female')
#stroke_bmi = stroke_bmi.rename(columns={"draw_0": "stroke_bmi_rr"}).reset_index()
stroke_bmi = stroke_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_bmi.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,157,370,9031,103,2019,14,2,495,1,...,1.622499,1.664811,1.815005,1.717803,1.767093,1.593233,1.685438,417614,45,Female
1,1,158,370,9031,103,2019,15,2,495,1,...,1.502447,1.658543,1.619314,1.529554,1.636810,1.555636,1.650060,417614,50,Female
2,2,159,370,9031,103,2019,13,2,495,1,...,1.771506,1.986334,1.630864,1.771523,1.798439,1.710487,1.774601,417614,40,Female
3,3,160,370,9031,103,2019,18,2,495,1,...,1.343073,1.313164,1.347364,1.307637,1.477685,1.416012,1.357726,417614,65,Female
4,4,161,370,9031,103,2019,10,2,495,1,...,1.537982,3.316202,2.051157,3.284436,2.284772,2.609347,1.742367,417614,25,Female


In [4]:
ihd_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_fpg = ihd_fpg.loc[ihd_fpg.cause_id==493].reset_index()
#ihd_fpg = ihd_fpg[['age_group_id','sex_id','draw_0']]
ihd_fpg['age_start'] = np.where(ihd_fpg['age_group_id']==10,25,np.where(ihd_fpg['age_group_id']==11,30,np.where(ihd_fpg['age_group_id']==12,35,np.where(ihd_fpg['age_group_id']==13,40,np.where(ihd_fpg['age_group_id']==14,45,np.where(ihd_fpg['age_group_id']==15,50,np.where(ihd_fpg['age_group_id']==16,55,np.where(ihd_fpg['age_group_id']==17,60,np.where(ihd_fpg['age_group_id']==18,65,np.where(ihd_fpg['age_group_id']==19,70,np.where(ihd_fpg['age_group_id']==20,75,np.where(ihd_fpg['age_group_id']==30,80,np.where(ihd_fpg['age_group_id']==31,85,np.where(ihd_fpg['age_group_id']==32,90,0))))))))))))))
ihd_fpg['sex'] = np.where(ihd_fpg['sex_id']==1,'Male','Female')
#ihd_fpg = ihd_fpg.rename(columns={"draw_0": "ihd_fpg_rr"}).reset_index()
ihd_fpg = ihd_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_fpg.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,0,105,9056,1,2019,10,2,493,1,...,1.2452,1.2914,1.6246,1.6488,1.4643,1.5122,1.4424,462929,25,Female
1,1,1,105,9056,1,2019,11,2,493,1,...,1.2468,1.2081,1.4179,1.4272,1.3878,1.3433,1.2971,462929,30,Female
2,2,2,105,9056,1,2019,12,2,493,1,...,1.2483,1.1247,1.2113,1.2058,1.3112,1.1744,1.1517,462929,35,Female
3,3,3,105,9056,1,2019,13,2,493,1,...,1.2440,1.0951,1.1069,1.0977,1.2680,1.1004,1.0913,462929,40,Female
4,4,4,105,9056,1,2019,14,2,493,1,...,1.2340,1.1193,1.1048,1.1030,1.2580,1.1213,1.1157,462929,45,Female


In [5]:
stroke_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_fpg = stroke_fpg.loc[stroke_fpg.cause_id==495].reset_index()
#stroke_fpg = stroke_fpg[['age_group_id','sex_id','draw_0']]
stroke_fpg['age_start'] = np.where(stroke_fpg['age_group_id']==10,25,np.where(stroke_fpg['age_group_id']==11,30,np.where(stroke_fpg['age_group_id']==12,35,np.where(stroke_fpg['age_group_id']==13,40,np.where(stroke_fpg['age_group_id']==14,45,np.where(stroke_fpg['age_group_id']==15,50,np.where(stroke_fpg['age_group_id']==16,55,np.where(stroke_fpg['age_group_id']==17,60,np.where(stroke_fpg['age_group_id']==18,65,np.where(stroke_fpg['age_group_id']==19,70,np.where(stroke_fpg['age_group_id']==20,75,np.where(stroke_fpg['age_group_id']==30,80,np.where(stroke_fpg['age_group_id']==31,85,np.where(stroke_fpg['age_group_id']==32,90,0))))))))))))))
stroke_fpg['sex'] = np.where(stroke_fpg['sex_id']==1,'Male','Female')
#stroke_fpg = stroke_fpg.rename(columns={"draw_0": "stroke_fpg_rr"}).reset_index()
stroke_fpg = stroke_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_fpg.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,22,105,9056,1,2019,10,2,495,1,...,1.3231,1.1216,1.2407,1.1612,1.5519,1.6935,1.8770,462929,25,Female
1,1,23,105,9056,1,2019,11,2,495,1,...,1.3219,1.1069,1.2508,1.1223,1.4916,1.5245,1.6430,462929,30,Female
2,2,24,105,9056,1,2019,12,2,495,1,...,1.3206,1.0923,1.2609,1.0834,1.4312,1.3555,1.4090,462929,35,Female
3,3,25,105,9056,1,2019,13,2,495,1,...,1.3108,1.0940,1.2630,1.0720,1.3804,1.2641,1.2833,462929,40,Female
4,4,26,105,9056,1,2019,14,2,495,1,...,1.2923,1.1120,1.2570,1.0880,1.3392,1.2502,1.2658,462929,45,Female


In [23]:
ihd_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_sbp = ihd_sbp.loc[ihd_sbp.cause_id==493].reset_index()
#ihd_sbp = ihd_sbp[['age_group_id','sex_id','draw_0']]
ihd_sbp['age_start'] = np.where(ihd_sbp['age_group_id']==10,25,np.where(ihd_sbp['age_group_id']==11,30,np.where(ihd_sbp['age_group_id']==12,35,np.where(ihd_sbp['age_group_id']==13,40,np.where(ihd_sbp['age_group_id']==14,45,np.where(ihd_sbp['age_group_id']==15,50,np.where(ihd_sbp['age_group_id']==16,55,np.where(ihd_sbp['age_group_id']==17,60,np.where(ihd_sbp['age_group_id']==18,65,np.where(ihd_sbp['age_group_id']==19,70,np.where(ihd_sbp['age_group_id']==20,75,np.where(ihd_sbp['age_group_id']==30,80,np.where(ihd_sbp['age_group_id']==31,85,np.where(ihd_sbp['age_group_id']==32,90,0))))))))))))))
ihd_sbp['sex'] = np.where(ihd_sbp['sex_id']==1,'Male','Female')
#ihd_sbp = ihd_sbp.rename(columns={"draw_0": "ihd_sbp_rr"}).reset_index()
ihd_sbp = ihd_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_sbp.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,55,107,9030,1,2019,10,1,493,1,...,1.4636,1.5379,1.5368,1.5914,1.3202,1.5269,1.8140,417554,25,Male
1,1,56,107,9030,1,2019,11,1,493,1,...,1.4914,1.5651,1.5517,1.5901,1.3577,1.5501,1.7720,417554,30,Male
2,2,57,107,9030,1,2019,12,1,493,1,...,1.5191,1.5924,1.5666,1.5887,1.3952,1.5734,1.7300,417554,35,Male
3,3,58,107,9030,1,2019,13,1,493,1,...,1.5178,1.5805,1.5552,1.5677,1.4101,1.5708,1.6785,417554,40,Male
4,4,59,107,9030,1,2019,14,1,493,1,...,1.4875,1.5295,1.5175,1.5270,1.4022,1.5425,1.6175,417554,45,Male


In [7]:
stroke_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_sbp = stroke_sbp.loc[stroke_sbp.cause_id==495].reset_index()
#stroke_sbp = stroke_sbp[['age_group_id','sex_id','draw_0']]
stroke_sbp['age_start'] = np.where(stroke_sbp['age_group_id']==10,25,np.where(stroke_sbp['age_group_id']==11,30,np.where(stroke_sbp['age_group_id']==12,35,np.where(stroke_sbp['age_group_id']==13,40,np.where(stroke_sbp['age_group_id']==14,45,np.where(stroke_sbp['age_group_id']==15,50,np.where(stroke_sbp['age_group_id']==16,55,np.where(stroke_sbp['age_group_id']==17,60,np.where(stroke_sbp['age_group_id']==18,65,np.where(stroke_sbp['age_group_id']==19,70,np.where(stroke_sbp['age_group_id']==20,75,np.where(stroke_sbp['age_group_id']==30,80,np.where(stroke_sbp['age_group_id']==31,85,np.where(stroke_sbp['age_group_id']==32,90,0))))))))))))))
stroke_sbp['sex'] = np.where(stroke_sbp['sex_id']==1,'Male','Female')
#stroke_sbp = stroke_sbp.rename(columns={"draw_0": "stroke_sbp_rr"}).reset_index()
stroke_sbp = stroke_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_sbp.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,110,107,9030,1,2019,10,1,495,1,...,1.6336,2.0695,2.5927,2.4871,1.9611,2.0101,1.6038,417554,25,Male
1,1,111,107,9030,1,2019,11,1,495,1,...,1.6209,1.9665,2.1593,2.0359,1.7894,1.8429,1.6047,417554,30,Male
2,2,112,107,9030,1,2019,12,1,495,1,...,1.6083,1.8635,1.7257,1.5846,1.6178,1.6756,1.6056,417554,35,Male
3,3,113,107,9030,1,2019,13,1,495,1,...,1.5825,1.7853,1.4954,1.3662,1.5208,1.5876,1.5750,417554,40,Male
4,4,114,107,9030,1,2019,14,1,495,1,...,1.5435,1.7318,1.4682,1.3805,1.4985,1.5788,1.5130,417554,45,Male


In [8]:
ihd_ldl = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=367, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_ldl = ihd_ldl.loc[ihd_ldl.cause_id==493].reset_index()
#ihd_ldl = ihd_ldl[['age_group_id','sex_id','draw_0']]
ihd_ldl['age_start'] = np.where(ihd_ldl['age_group_id']==10,25,np.where(ihd_ldl['age_group_id']==11,30,np.where(ihd_ldl['age_group_id']==12,35,np.where(ihd_ldl['age_group_id']==13,40,np.where(ihd_ldl['age_group_id']==14,45,np.where(ihd_ldl['age_group_id']==15,50,np.where(ihd_ldl['age_group_id']==16,55,np.where(ihd_ldl['age_group_id']==17,60,np.where(ihd_ldl['age_group_id']==18,65,np.where(ihd_ldl['age_group_id']==19,70,np.where(ihd_ldl['age_group_id']==20,75,np.where(ihd_ldl['age_group_id']==30,80,np.where(ihd_ldl['age_group_id']==31,85,np.where(ihd_ldl['age_group_id']==32,90,0))))))))))))))
ihd_ldl['sex'] = np.where(ihd_ldl['sex_id']==1,'Male','Female')
#ihd_ldl = ihd_ldl.rename(columns={"draw_0": "ihd_ldl_rr"}).reset_index()
ihd_ldl = ihd_ldl.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_ldl.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,0,367,18824,1,2019,10,1,493,1,...,1.8168,1.7514,1.8246,2.4743,1.9976,1.8399,2.1089,417545,25,Male
1,1,1,367,18824,1,2019,11,1,493,1,...,1.9253,1.8676,1.8559,2.3378,2.0169,1.9631,2.1261,417545,30,Male
2,2,2,367,18824,1,2019,12,1,493,1,...,2.0337,1.9839,1.8873,2.2012,2.0363,2.0864,2.1434,417545,35,Male
3,3,3,367,18824,1,2019,13,1,493,1,...,2.0100,1.9803,1.8422,2.0569,1.9709,2.0628,2.0688,417545,40,Male
4,4,4,367,18824,1,2019,14,1,493,1,...,1.8540,1.8570,1.7205,1.9047,1.8207,1.8925,1.9023,417545,45,Male


In [9]:
stroke_ldl = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=367, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_ldl = stroke_ldl.loc[stroke_ldl.cause_id==495].reset_index()
#stroke_ldl = stroke_ldl[['age_group_id','sex_id','draw_0']]
stroke_ldl['age_start'] = np.where(stroke_ldl['age_group_id']==10,25,np.where(stroke_ldl['age_group_id']==11,30,np.where(stroke_ldl['age_group_id']==12,35,np.where(stroke_ldl['age_group_id']==13,40,np.where(stroke_ldl['age_group_id']==14,45,np.where(stroke_ldl['age_group_id']==15,50,np.where(stroke_ldl['age_group_id']==16,55,np.where(stroke_ldl['age_group_id']==17,60,np.where(stroke_ldl['age_group_id']==18,65,np.where(stroke_ldl['age_group_id']==19,70,np.where(stroke_ldl['age_group_id']==20,75,np.where(stroke_ldl['age_group_id']==30,80,np.where(stroke_ldl['age_group_id']==31,85,np.where(stroke_ldl['age_group_id']==32,90,0))))))))))))))
stroke_ldl['sex'] = np.where(stroke_ldl['sex_id']==1,'Male','Female')
#stroke_ldl = stroke_ldl.rename(columns={"draw_0": "stroke_ldl_rr"}).reset_index()
stroke_ldl = stroke_ldl.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_ldl.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,cause_id,mortality,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0,11,367,18824,1,2019,10,1,495,1,...,1.3991,1.6262,1.3285,1.4406,1.7056,1.7891,1.6429,417545,25,Male
1,1,12,367,18824,1,2019,11,1,495,1,...,1.3974,1.6653,1.3515,1.3719,1.6674,1.7034,1.5101,417545,30,Male
2,2,13,367,18824,1,2019,12,1,495,1,...,1.3958,1.7044,1.3745,1.3033,1.6291,1.6178,1.3774,417545,35,Male
3,3,14,367,18824,1,2019,13,1,495,1,...,1.3648,1.6732,1.3596,1.2497,1.5613,1.5318,1.2913,417545,40,Male
4,4,15,367,18824,1,2019,14,1,495,1,...,1.3045,1.5715,1.3068,1.2112,1.4640,1.4455,1.2517,417545,45,Male


### Now we can calculate the deltas
Note that we are finding the same value (deltas) based on two causes (IHD and stroke). Ideally, these should be identical. We find that they are very similar, making me more confident in these calculations.

In [10]:
sbp_ihd_mf = 0.31
ldl_ihd_mf = 0.10
fpg_ihd_mf = 0.15

sbp_stroke_mf = 0.65
ldl_stroke_mf = 0.03
fpg_stroke_mf = 0.22

In [33]:
bmi_delta = ihd_bmi.merge(ihd_fpg[['ihd_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    ihd_sbp[['ihd_sbp_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_bmi[['stroke_bmi_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_fpg[['stroke_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_ldl[['stroke_ldl_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_sbp[['stroke_sbp_rr','age_start','sex']], on=['age_start','sex'])

bmi_delta['fpg_delta_1'] = np.log((fpg_ihd_mf*(bmi_delta['ihd_bmi_rr']-1))+1)/np.log(bmi_delta['ihd_fpg_rr'])
bmi_delta['fpg_delta_2'] = np.log((fpg_stroke_mf*(bmi_delta['stroke_bmi_rr']-1))+1)/np.log(bmi_delta['stroke_fpg_rr'])
bmi_delta['sbp_delta_1'] = np.log((sbp_ihd_mf*(bmi_delta['ihd_bmi_rr']-1))+1)/np.log(bmi_delta['ihd_sbp_rr'])
bmi_delta['sbp_delta_2'] = np.log((sbp_stroke_mf*(bmi_delta['stroke_bmi_rr']-1))+1)/np.log(bmi_delta['stroke_sbp_rr'])

bmi_delta['delta_diff_fpg'] = bmi_delta['fpg_delta_1'] - bmi_delta['fpg_delta_2']
bmi_delta['delta_diff_sbp'] = bmi_delta['sbp_delta_1'] - bmi_delta['sbp_delta_2']

bmi_delta = bmi_delta[['ihd_bmi_rr','ihd_fpg_rr','ihd_sbp_rr','stroke_bmi_rr','stroke_sbp_rr','stroke_fpg_rr','age_start', 'sex','fpg_delta_1','sbp_delta_1', 'fpg_delta_2','sbp_delta_2','delta_diff_fpg','delta_diff_sbp']]
bmi_delta

,ihd_bmi_rr,ihd_fpg_rr,ihd_sbp_rr,stroke_bmi_rr,stroke_sbp_rr,stroke_fpg_rr,age_start,sex,fpg_delta_1,sbp_delta_1,fpg_delta_2,sbp_delta_2,delta_diff_fpg,delta_diff_sbp
0,1.591258,1.2805,1.5348,1.809969,1.5435,1.2012,45,Female,0.343675,0.392854,0.894507,0.974454,-0.550832,-0.581600
1,1.379902,1.1815,1.4203,1.497946,1.4105,1.1877,60,Female,0.332289,0.317314,0.604311,0.815261,-0.272022,-0.497947
2,1.685112,1.4505,1.6799,1.919950,1.6409,1.2711,35,Female,0.263028,0.371268,0.768336,0.946466,-0.505308,-0.575199
3,1.665879,1.3135,1.5729,1.738278,1.5878,1.2058,40,Female,0.349118,0.414331,0.804223,0.847760,-0.455106,-0.433429
4,1.291149,1.1897,1.2947,1.273751,1.2820,1.1660,75,Female,0.246087,0.334571,0.380790,0.659225,-0.134703,-0.324654
5,1.296208,1.1512,1.3402,1.271957,1.3233,1.1760,70,Female,0.308742,0.300015,0.358435,0.581073,-0.049693,-0.281058
6,2.094858,1.6915,1.8558,2.220086,1.7026,1.3974,30,Female,0.289297,0.472616,0.710585,1.097274,-0.421288,-0.624657
7,3.510608,1.9325,2.0317,2.405075,1.7644,1.5236,25,Female,0.485129,0.812066,0.639677,1.142685,-0.154547,-0.330619
8,1.382845,1.2475,1.4966,1.727334,1.4992,1.1968,50,Female,0.252501,0.278156,0.826221,0.956070,-0.573720,-0.677915
9,1.425974,1.2145,1.4584,1.556205,1.4548,1.1923,55,Female,0.318720,0.328701,0.656330,0.823253,-0.337610,-0.494552


In [12]:
print(bmi_delta['delta_diff_sbp'].mean())
print(bmi_delta['delta_diff_sbp'].min())
print(bmi_delta['delta_diff_fpg'].mean())
print(bmi_delta['delta_diff_fpg'].min())

-0.37932985661960256
-0.6779145317209673
-0.25589677366874863
-0.5737199459174113


The deltas here are quite different now... which is a bit concerning to our approach. I think using the IHD delta values makes the most sense as IHD is more likely to be similar to HF. 

### The RR for SBP to HF is categorical, we need continuous
The literature value for SBP RR to HF are in categories. To account for this, we found a continuous RR that approximates the same values. The mean_SBP are based on exposure values form GBD.

In [13]:
SBP_RR = pd.DataFrame(np.array([['<120',1,110.6],['120-129',1.27,124.4],['130-139',1.5,134.2],['140+',1.76,150.7]]),
                   columns=['SBP_Group', 'RR_original', 'Mean_SBP'])
SBP_RR['Mean_SBP'] = pd.to_numeric(SBP_RR['Mean_SBP'])
SBP_RR['Change_in_SBP'] = SBP_RR['Mean_SBP'] - 110.6
cont_RR = 1.016
SBP_RR['New_RR'] = np.power(cont_RR,SBP_RR['Change_in_SBP'])
SBP_RR

,SBP_Group,RR_original,Mean_SBP,Change_in_SBP,New_RR
0,<120,1,110.6,0.0,1.000000
1,120-129,1.27,124.4,13.8,1.244896
2,130-139,1.5,134.2,23.6,1.454426
3,140+,1.76,150.7,40.1,1.889895


### Lastly, we find the Mediation Factors

In [14]:
rr_sbp = 1.016
rr_bmi = 1.14

In [34]:
bmi_delta['sbp_delta_1'] = bmi_delta.sbp_delta_1 * 2 
bmi_delta = bmi_delta[['age_start', 'sex', 'sbp_delta_1']]
bmi_delta#.head()

,age_start,sex,sbp_delta_1
0,45,Female,0.785708
1,60,Female,0.634628
2,35,Female,0.742535
3,40,Female,0.828662
4,75,Female,0.669142
5,70,Female,0.600031
6,30,Female,0.945233
7,25,Female,1.624132
8,50,Female,0.556311
9,55,Female,0.657401


In [32]:
HF_MF=pd.DataFrame(bmi_delta)
HF_MF.to_csv('/ihme/homes/lutzes/heart_failure_MF.csv')

### Testing based on the intervention if we need FPG
This is bit tough/estimated since right now I am extracting data from a figure. I hope to find better data at some point. I am choosing to assess this at 1 year since that is when there is the greatest change in both BMI and FPG.

In [17]:
#int_bmi = 2.5
#int_fpg = 0.3 mmol/L --> 5 mg/dL

For BMI will account for a 1/2 unit change in FPG, the intervention saw a ~5 unit change, we should add mediation

## New Testing Based on Zeb's Calculations

In [18]:
bmi_ihd = ihd_bmi.loc[(ihd_bmi.age_start==70) & (ihd_bmi.sex == 'Female')].ihd_bmi_rr.values[0]
sbp_ihd = ihd_sbp.loc[(ihd_sbp.age_start==70) & (ihd_sbp.sex == 'Female')].ihd_sbp_rr.values[0]
ldl_ihd = ihd_ldl.loc[(ihd_ldl.age_start==70) & (ihd_ldl.sex == 'Female')].ihd_ldl_rr.values[0]
fpg_ihd = ihd_fpg.loc[(ihd_fpg.age_start==70) & (ihd_fpg.sex == 'Female')].ihd_fpg_rr.values[0]

sbp_mf = 0.31
ldl_mf = 0.10
fpg_mf = 0.15
print(bmi_ihd,sbp_ihd,ldl_ihd,fpg_ihd)

1.296208 1.3401999 1.2347 1.1512001


In [19]:
## Alternative way 
## Δ_(r→m)=(log⁡(MF∗(RR_(R→O,total)−1)+1))/(log⁡(RR_(M→O)))
delta_sbp = np.log((sbp_mf*(bmi_ihd-1))+1)/np.log(sbp_ihd)
delta_ldl = np.log(ldl_mf*(bmi_ihd-1)+1)/np.log(ldl_ihd)
delta_fpg = np.log(fpg_mf*(bmi_ihd-1)+1)/np.log(fpg_ihd)
print(delta_sbp,delta_ldl,delta_fpg)

0.3000153583753123 0.13845682533912737 0.308742177413229


In [20]:
## Method used today
## RR adj = 1+ (RR unad -1) * (1-MF_SBP) * (1-MF_LDL) * (1-MF_FPG)
RR_adj_1 = 1+(bmi_ihd -1)*(1-sbp_mf)*(1-ldl_mf)*(1-fpg_mf)
RR_adj_1

1.1563533928

In [21]:
RR_adj_1 * ((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))

1.3576897859933077

In [22]:
## RR_adj=(BMI_IHD)/(∏_m▒(RR_(m→o) )^(Δ_(r→m)))
RR_adj_2=(bmi_ihd)/((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))
RR_adj_2

1.1039889480187124

In [23]:
RR_adj_2 * ((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))

1.296208

### Also for Stroke

In [24]:
bmi_stroke = stroke_bmi.loc[(stroke_bmi.age_start==70) & (stroke_bmi.sex == 'Female')].stroke_bmi_rr.values[0]
sbp_stroke = stroke_sbp.loc[(stroke_sbp.age_start==70) & (stroke_sbp.sex == 'Female')].stroke_sbp_rr.values[0]
ldl_stroke = stroke_ldl.loc[(stroke_ldl.age_start==70) & (stroke_ldl.sex == 'Female')].stroke_ldl_rr.values[0]
fpg_stroke = stroke_fpg.loc[(stroke_fpg.age_start==70) & (stroke_fpg.sex == 'Female')].stroke_fpg_rr.values[0]

sbp_mf = 0.65
ldl_mf = 0.03
fpg_mf = 0.22
print(bmi_stroke,sbp_ihd,ldl_ihd,fpg_ihd)

1.271957 1.3401999 1.2347 1.1512001


In [25]:
## Alternative way 
## Δ_(r→m)=(log⁡(MF∗(RR_(R→O,total)−1)+1))/(log⁡(RR_(M→O)))
delta_sbp = np.log((sbp_mf*(bmi_stroke-1))+1)/np.log(sbp_stroke)
delta_ldl = np.log(ldl_mf*(bmi_stroke-1)+1)/np.log(ldl_stroke)
delta_fpg = np.log(fpg_mf*(bmi_stroke-1)+1)/np.log(fpg_stroke)
print(delta_sbp,delta_ldl,delta_fpg)

0.5810728694359281 0.08123462688345602 0.3584347385074138


In [26]:
## Method used today
## RR adj = 1+ (RR unad -1) * (1-MF_SBP) * (1-MF_LDL) * (1-MF_FPG)
RR_adj_1 = 1+(bmi_stroke -1)*(1-sbp_mf)*(1-ldl_mf)*(1-fpg_mf)
RR_adj_1

1.07201693317

In [27]:
RR_adj_1 * ((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))

1.347905131294933

In [28]:
## RR_adj=(BMI_IHD)/(∏_m▒(RR_(m→o) )^(Δ_(r→m)))
RR_adj_2=(bmi_stroke)/((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))
RR_adj_2

1.0116138076825494

In [29]:
RR_adj_2 * ((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))

1.271957

## Finding the Draw Level Deltas for BMI through SBP

In [36]:
mfs = pd.read_csv('/mnt/team/simulation_science/costeffectiveness/artifacts/vivarium_nih_us_cvd/raw_data/mediation_matrix_draw_gbd_2021_edited.csv')
mfs = mfs.loc[(mfs.rei_id == 370) & (mfs.med_id == 107) & (mfs.cause_id == 493)]
mfs

,rei_id,med_id,cause_id,mean_mediation,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
81,370,107,493,0.311516,0.28805,0.333642,0.308331,0.336447,0.302602,0.313664,...,0.306973,0.325752,0.284115,0.306329,0.326376,0.348663,0.300246,0.325386,0.335548,0.307723


In [39]:
mfs.iloc[:,4:1004]

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
81,0.28805,0.333642,0.308331,0.336447,0.302602,0.313664,0.336983,0.340481,0.325711,0.303146,...,0.306973,0.325752,0.284115,0.306329,0.326376,0.348663,0.300246,0.325386,0.335548,0.307723


In [69]:
sbp_rr = ihd_sbp.drop(columns=['level_0','index','rei_id','modelable_entity_id','location_id','year_id',
                                'age_group_id','cause_id','sex_id','mortality','morbidity','metric_id','parameter','exposure'])
bmi_rr = ihd_bmi.drop(columns=['level_0','index','rei_id','modelable_entity_id','location_id','year_id',
                                'age_group_id','cause_id','sex_id','mortality','morbidity','metric_id','parameter','exposure'])
sbp_rr.iloc[:,:1000].head()

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,2.0317,1.8064,1.5646,1.8926,2.0621,2.5817,2.5088,2.7671,2.5276,2.1318,...,1.9449,1.4464,1.3839,1.4636,1.5379,1.5368,1.5914,1.3202,1.5269,1.8140
1,1.8558,1.7631,1.5619,1.8579,1.8324,2.2023,2.1733,2.3999,2.2229,1.9347,...,1.7481,1.4546,1.4251,1.4914,1.5651,1.5517,1.5901,1.3577,1.5501,1.7720
2,1.6799,1.7197,1.5593,1.8233,1.6028,1.8228,1.8377,2.0326,1.9183,1.7376,...,1.5514,1.4629,1.4664,1.5191,1.5924,1.5666,1.5887,1.3952,1.5734,1.7300
3,1.5729,1.6641,1.5396,1.7738,1.4928,1.6184,1.6465,1.8081,1.7283,1.6162,...,1.4472,1.4605,1.4749,1.5178,1.5805,1.5552,1.5677,1.4101,1.5708,1.6785
4,1.5348,1.5962,1.5028,1.7092,1.5025,1.5892,1.5995,1.7262,1.6530,1.5705,...,1.4355,1.4475,1.4508,1.4875,1.5295,1.5175,1.5270,1.4022,1.5425,1.6175


In [68]:
#Formula delta = (log((MF * (BMI RR - 1)) +1) / log(SBP RR)) ^2

bmi_rr.iloc[:,:1000] = (bmi_rr.iloc[:,:1000] - 1)
bmi_rr.iloc[:,:1000] = pd.DataFrame(bmi_rr.iloc[:,:1000].values*mfs.iloc[:,4:1004].values, 
                       columns=bmi_rr.iloc[:,:1000].columns, index=bmi_rr.iloc[:,:1000].index)

bmi_rr.iloc[:,:1000] = (bmi_rr.iloc[:,:1000] + 1)
bmi_rr.iloc[:,:1000] = np.log(bmi_rr.iloc[:,:1000])

sbp_rr.iloc[:,:1000] = np.log(sbp_rr.iloc[:,:1000])

bmi_rr.iloc[:,:1000] = pd.DataFrame(bmi_rr.iloc[:,:1000].values / sbp_rr.iloc[:,:1000].values, 
                       columns=bmi_rr.iloc[:,:1000].columns, index=bmi_rr.iloc[:,:1000].index)

bmi_rr.iloc[:,:1000] = (bmi_rr.iloc[:,:1000])**2

bmi_rr#.head()
##Spot checked this and seems to be working as I expected it to 

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,model_version_id,age_start,sex
0,0.049222,0.091738,0.106782,0.080371,0.037850,0.028047,0.045315,0.031814,0.031024,0.040945,...,0.193637,0.184972,0.150955,0.125081,0.428824,0.142195,0.072051,417614,45,Female
1,0.028208,0.040947,0.075854,0.036097,0.032363,0.021759,0.029347,0.024392,0.017182,0.041700,...,0.128223,0.084291,0.103442,0.069550,0.153877,0.096222,0.056826,417614,60,Female
2,0.120553,0.171240,0.152497,0.132208,0.131390,0.171153,0.112264,0.078699,0.089370,0.142530,...,0.129124,0.201961,0.171753,0.140778,0.292081,0.176570,0.105817,417614,35,Female
3,0.150094,0.110266,0.165291,0.084084,0.217535,0.093225,0.110418,0.087891,0.118581,0.088050,...,0.213279,0.129352,0.111830,0.101389,0.281850,0.263850,0.095155,417614,40,Female
4,0.035339,0.044651,0.024764,0.021816,0.063511,0.031157,0.049754,0.025305,0.021850,0.025168,...,0.021299,0.023695,0.062870,0.037009,0.065043,0.040252,0.029953,417614,75,Female
5,0.041238,0.086378,0.045586,0.037317,0.057465,0.037184,0.049319,0.051424,0.052283,0.069422,...,0.059703,0.044190,0.089496,0.048028,0.068786,0.036695,0.051904,417614,70,Female
6,0.527740,0.432387,1.952304,0.418420,0.127442,0.245270,0.551437,0.843508,0.199828,0.935732,...,0.578600,1.237753,0.702354,1.025209,0.401026,0.153808,0.660458,417614,30,Female
7,2.405387,0.444181,0.493946,0.314497,0.295611,0.355222,0.044617,0.316616,0.431209,0.470781,...,0.849105,2.593429,1.117598,2.141064,0.766883,0.248043,1.860658,417614,25,Female
8,0.104503,0.320549,0.233306,0.161519,0.116819,0.153807,0.213433,0.266628,0.265952,0.187506,...,0.271785,0.309784,0.290756,0.255335,0.230081,0.243775,0.177811,417614,50,Female
9,0.156227,0.233025,0.277003,0.229324,0.076821,0.153021,0.188858,0.250907,0.253619,0.222047,...,0.227964,0.335914,0.288328,0.182669,0.259239,0.156807,0.268761,417614,55,Female


In [70]:
ihd_bmi_small.to_csv('/ihme/homes/lutzes/heart_failure_deltas_all_draws.csv')